套件導入

In [1]:
import pandas as pd 
import numpy as np 
import tejapi


In [2]:
api_key = 'your_api_key'
tejapi.ApiConfig.api_key = api_key
tejapi.ApiConfig.ignoretz = True

資料庫導入

In [18]:
comp_data = tejapi.get('TWN/ANPRCSTD',
                   paginate = True,
                  opts = {"columns":["coid", "mdate", "mkt", "stype", "list_date", "delis_date", "tseind"]}
                  )

In [22]:
coid_lst = list(comp_data.loc[(comp_data["stype"] == "STOCK") & (comp_data["tseind"] == "22")]["coid"])

In [24]:
len(coid_lst)

241

In [70]:
gte, lte = "2020-01-01", "2022-12-31"
price_data = tejapi.get('TWN/AAPRCDA',
                        paginate = True,
                        coid = coid_lst,
                        mdate = {"gte":gte, "lte":lte},
                        opts = {"columns":["coid", "mdate", "fld014", "cls60", "close_d"]}
                  )

In [26]:
# price_data = pd.read_csv("price_data.csv")

資料前處理

In [71]:
price_data["coid"] = price_data["coid"].astype(str)

In [72]:
inter_coid = list(set(coid_lst).intersection(set(price_data["coid"].unique())))

In [73]:
inter_coid

['8432',
 '6645',
 '1760',
 '4195',
 '6661',
 '1733',
 '8436',
 '4155',
 '6872',
 '6493',
 '4114',
 '1799',
 '1783',
 '4167',
 '4126',
 '6549',
 '6885',
 '4736',
 '6586',
 '6703',
 '4911',
 '3218',
 '6580',
 '6518',
 '7595',
 '6637',
 '4129',
 '4131',
 '5312',
 '4771',
 '6572',
 '6242',
 '4151',
 '6747',
 '6527',
 '6767',
 '6535',
 '4138',
 '6612',
 '1762',
 '3176',
 '1736',
 '6130',
 '6794',
 '4116',
 '8403',
 '7575',
 '6589',
 '1789',
 '6838',
 '1598',
 '4164',
 '4108',
 '1271',
 '4105',
 '6562',
 '6543',
 '6861',
 '6762',
 '6891',
 '4166',
 '6677',
 '6610',
 '4162',
 '4175',
 '6472',
 '6492',
 '6665',
 '6496',
 '4109',
 '6758',
 '6844',
 '6879',
 '1777',
 '6461',
 '6748',
 '6929',
 '4106',
 '6734',
 '6932',
 '4119',
 '3164',
 '6569',
 '7427',
 '4732',
 '4127',
 '4728',
 '6657',
 '1786',
 '4194',
 '6566',
 '6857',
 '4183',
 '4133',
 '7561',
 '4192',
 '4743',
 '6850',
 '1784',
 '6469',
 '4173',
 '4170',
 '6547',
 '4161',
 '6633',
 '4197',
 '4735',
 '6848',
 '3118',
 '1780',
 '4130',
 

In [186]:
price_data[price_data["coid"].isin(inter_coid)]

,coid,mdate,fld014,cls60,close_d
None,,,,,
0,1271,2021-01-18,70.30,70.3000,70.3
1,1271,2021-01-19,69.70,69.7000,69.1
2,1271,2021-01-20,69.60,69.6000,69.4
3,1271,2021-01-21,69.10,69.1000,67.6
4,1271,2021-01-22,69.12,69.1200,69.2
...,...,...,...,...,...
126739,8436,2022-12-26,165.75,132.6083,175.0
126740,8436,2022-12-27,167.75,133.5083,177.5
126741,8436,2022-12-28,168.65,134.2917,171.0


交易策略

In [209]:
fltr_price_data = price_data[price_data["coid"].isin(inter_coid)]
group = fltr_price_data.groupby("coid")


In [210]:
def MA_strategy(df, principal):
    position = 0 
    lst = []
    
    for i in range(len(df)):
        if df["fld014"].iloc[i] > df["cls60"].iloc[i] and principal >= float(df["close_d"].iloc[i])*1*1000:
            principal -= float(df["close_d"].iloc[i])*1*1000
            position += 1
            lst.append({
                "日期": df["mdate"].iloc[i],
                "標的": df["coid"].iloc[i],
                "買入/賣出": "買入",
                "單價": float(df["close_d"].iloc[i]),
                "單位": 1,
                "剩餘現金": principal,
                "部位" : position
            })

        elif df["fld014"].iloc[i] < df["cls60"].iloc[i] and position > 0:
            principal += float(df["close_d"].iloc[i])*1*1000
            position -= 1
            
            lst.append({
                "日期": df["mdate"].iloc[i],
                "標的": df["coid"].iloc[i],
                "買入/賣出": "賣出",
                "單價": float(df["close_d"].iloc[i]),
                "單位": 1,
                "剩餘現金": principal,
                "部位" : position
            })
            
        elif i == len(df)-1 and position > 0:
            principal += float(df["close_d"].iloc[i])*position*1000
            position -= position
            lst.append({
                "日期": df["mdate"].iloc[i],
                "標的": df["coid"].iloc[i],
                "買入/賣出": "賣出",
                "單價": float(df["close_d"].iloc[i]),
                "單位": position,
                "剩餘現金": principal,
                "部位" : position
            })
            

    df_output = pd.DataFrame(lst)
    
    
    
    return (df_output, principal)
        


所有曾經上市櫃公司股價資訊

In [211]:
principal = 1000000
total_return_1 = 0
df = pd.DataFrame(columns = ["日期", "標的", "買入/賣出", "單價", "單位", "剩餘現金", "部位"])

for g in group.groups.keys():
    reuslt = MA_strategy(group.get_group(g), principal)
    total_return_1 += round(reuslt[1], 2)
    df = pd.concat([df, reuslt[0]], ignore_index=True)
    

In [212]:
total_return = (total_return_1/(1000000*len(group.groups.keys())) - 1) *100
print(f"ROI : {total_return}%")

-14.269384236453197


In [225]:
print(f'2020–01–01 至 2022–12–31狀態曾經為上市櫃的公司數量：{len(list(comp_data.loc[(comp_data["coid"].isin(inter_coid))]["coid"]))}')


2020–01–01 至 2022–12–31狀態曾經為上市櫃的公司數量：203


剔除下市櫃公司

In [213]:
without_dist_coid = comp_data.loc[(comp_data["coid"].isin(inter_coid)) & (comp_data["mkt"] != "DIST")]["coid"].unique()
fltr_price_data = price_data[price_data["coid"].isin(without_dist_coid)]
group = fltr_price_data.groupby("coid")

In [214]:
principal = 1000000
total_return_2 = 0
df_without_dist = pd.DataFrame(columns = ["日期", "標的", "買入/賣出", "單價", "單位", "剩餘現金", "部位"])

for g in group.groups.keys():
    reuslt = MA_strategy(group.get_group(g), principal)
    total_return_2 += round(reuslt[1], 2)
    df = pd.concat([df_without_dist, reuslt[0]], ignore_index=True)
    


In [226]:
return_without_dist = (total_return_2/(1000000*len(group.groups.keys())) - 1) *100
print(f"ROI : {return_without_dist}%")

ROI : -13.846355670103094%
